In [1]:
# install packages and libraries
%matplotlib inline
%matplotlib widget

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size

plt.rcParams["figure.figsize"] = (16, 9) #set default figure size (w, h) 
plt.style.use("ggplot")
import numpy.matlib
import copy
import scipy.sparse as sparse
from numpy.random import default_rng
import time

In [2]:
import sys # importing sys
  
# adding Latest_scripts to the system path
sys.path.insert(0, '../Latest_scripts/')
sys.path.insert(0, '../../../ExternalHP_Codes/SteveMorse/hawkes-master/')

import HP_scripts as HP # import module containing functions for the Masters project
import MHP as MHP # import module containing EM functions from Steve Morse for the Masters project

# Load previous session

In [4]:
import dill
#dill.dump_session('DJIA_multivHP_10_15_20_September_v2_random10.db')
dill.load_session('DJIA_multivHP_10_15_20_September_v1.db')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Load the data discussed and analysed in https://methods.sagepub.com/dataset/howtoguide/multivariate-hawkes-in-djia-2018#i100

In [3]:
# Read data
# Import the data as a dataframe (2D data structure with labelled axes)

df = pd.read_csv('financial_data/dataset-djia-2018-subset2.csv')
dates = pd.to_datetime(df['Date']) # set dates as the Date of closing price column


del df['Date'] # delete Date column

# Fill missing values
df.ffill(inplace=True)
df

,AABA,AAPL,AMZN,AXP,BA,CAT,CSCO,CVX,DIS,GE,...,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,WMT,XOM
0,40.91,10.68,47.58,52.58,70.44,57.80,17.45,59.08,24.40,35.37,...,26.84,10.74,23.78,58.78,45.99,61.73,56.53,30.38,46.23,58.47
1,40.97,10.71,47.25,51.95,71.17,59.27,17.85,58.91,23.99,35.32,...,26.97,10.69,24.55,58.89,46.50,61.88,56.19,31.27,46.32,58.57
2,41.53,10.63,47.65,52.50,70.33,59.27,18.35,58.19,24.41,35.23,...,26.99,10.76,24.58,58.70,46.95,61.69,55.98,31.63,45.69,58.28
3,43.21,10.90,47.87,52.68,69.35,60.45,18.77,59.25,24.74,35.47,...,26.91,10.72,24.85,58.64,47.21,62.90,56.16,31.35,45.88,59.43
4,43.42,10.86,47.08,53.99,68.77,61.55,19.06,58.95,25.00,35.38,...,26.86,10.88,24.85,59.08,47.23,61.40,56.80,31.48,45.71,59.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3015,71.58,175.01,1168.36,98.74,295.10,155.75,38.55,124.98,108.67,17.50,...,85.51,63.29,36.14,92.13,134.39,220.00,127.23,53.19,98.21,83.97
3016,69.86,170.57,1176.76,98.57,295.36,156.44,38.48,125.98,108.12,17.43,...,85.40,63.65,36.21,92.48,134.78,219.60,127.14,53.22,99.16,83.98
3017,70.06,170.60,1182.26,99.13,295.62,157.52,38.56,125.55,107.64,17.38,...,85.71,62.95,36.33,92.10,134.77,220.42,127.58,53.28,99.26,83.90
3018,69.82,171.08,1186.10,99.70,296.35,158.42,38.59,125.58,107.77,17.36,...,85.72,62.95,36.37,92.07,135.66,222.77,128.12,53.43,99.40,84.02


In [4]:
# Google and amazon are not in DJIA list

ten_companies = ['AAPL','MSFT', 'JPM', 'GS', 'PFE', 'MRK', 'NKE', 'HD',  'GOOGL','AMZN']

## Events: 10% price-drop

In [8]:
# Collect event times

t_i_10=[]
u_i_10=[] # collect marker for each event times

ticker_id = [] # collect integer marker of each stock, index correspodns to stock AABA, 1 correponds to stock AAPL
ticker = []
ticker_dict = {} # collect key-value pairs where key represents tikcer of a stock and it's corresponding value an integer marker


for u,col in enumerate(df[ten_companies]):
    Tdiff_10 = df[col].diff()
    timestamps_10=dates[Tdiff_10<=Tdiff_10.quantile(0.1)] # return timestamps where subsequent price difference were less than 10th percentile of the whole return series
    t_i_10.extend((timestamps_10 - pd.Timestamp(2006,1,3)).dt.days.astype(float)) # t_i contains timestamps of all extreme price-drops for all 
    u_i_10.extend(np.repeat(u,len(timestamps_10)))
    ticker_dict[col] = u
    ticker_id += [u]
    ticker += [col]


seed = 10
np.random.seed(seed)

t_i_10= (np.array(t_i_10))/30+np.random.rand(len(t_i_10))
u_i_10=np.array(u_i_10)
perm = np.argsort(t_i_10)
t_i_10 = t_i_10[perm] # superposed timestamps
u_i_10 = u_i_10[perm]

### Reshape timestamps and marker data into the form of Steve Morse's EM code

In [10]:
data10 = []
counter = 0
for t,u in zip(t_i_10, u_i_10):
    data10.append([])
    data10[counter] += [t, np.array([u], dtype='int')]
    counter += 1


data10 = np.array(data10)
data10

array([[0.43677219292984165, array([4])],
       [0.6543285601112814, array([8])],
       [0.8872564525116836, array([7])],
       ...,
       [146.40395017603421, array([4])],
       [146.49618469324798, array([9])],
       [146.73634087005024, array([0])]], dtype=object)

## Total number of observations for events of 10% price-drops were 

## Events: 15% price-drop

In [18]:
# Collect event times

t_i_15=[]
u_i_15=[] # collect marker for each event times

ticker_id = [] # collect integer marker of each stock, index correspodns to stock AABA, 1 correponds to stock AAPL
ticker = []
ticker_dict = {} # collect key-value pairs where key represents tikcer of a stock and it's corresponding value an integer marker


for u,col in enumerate(df[ten_companies]):
    Tdiff_15 = df[col].diff()
    timestamps_15=dates[Tdiff_15<=Tdiff_15.quantile(0.15)] # return timestamps where subsequent price difference were less than 10th percentile of the whole return series
    t_i_15.extend((timestamps_15 - pd.Timestamp(2006,1,3)).dt.days.astype(float)) # t_i contains timestamps of all extreme price-drops for all 
    u_i_15.extend(np.repeat(u,len(timestamps_15)))
    ticker_dict[col] = u
    ticker_id += [u]
    ticker += [col]


seed = 10
np.random.seed(seed)

t_i_15=(np.array(t_i_15))/30+np.random.rand(len(t_i_15))
u_i_15=np.array(u_i_15)
perm = np.argsort(t_i_15)
t_i_15 = t_i_15[perm] # superposed timestamps
u_i_15 = u_i_15[perm]

### Reshape timestamps and marker data into the form of Steve Morse's EM code

In [20]:
data15 = []
counter = 0
for t,u in zip(t_i_15, u_i_15):
    data15.append([])
    data15[counter] += [t, np.array([u], dtype='int')]
    counter += 1


data15 = np.array(data15)
data15.shape

(4540, 2)

## Total number of observations for events of 15% price-drops were 4540

## Events: 20% price-drop

In [21]:
# Collect event times

t_i=[]
u_i=[] # collect marker for each event times

ticker_id = [] # collect integer marker of each stock, index correspodns to stock AABA, 1 correponds to stock AAPL
ticker = []
ticker_dict = {} # collect key-value pairs where key represents tikcer of a stock and it's corresponding value an integer marker


for u,col in enumerate(df[ten_companies]):
    Tdiff = df[col].diff()
    timestamps=dates[Tdiff<=Tdiff.quantile(0.2)] # return timestamps where subsequent price difference were less than 10th percentile of the whole return series
    t_i.extend((timestamps - pd.Timestamp(2006,1,3)).dt.days.astype(float)) # t_i contains timestamps of all extreme price-drops for all 
    u_i.extend(np.repeat(u,len(timestamps)))
    ticker_dict[col] = u
    ticker_id += [u]
    ticker += [col]


seed = 10
np.random.seed(seed)
t_i=(np.array(t_i))/30+np.random.rand(len(t_i))
u_i=np.array(u_i)
perm = np.argsort(t_i)
t_i = t_i[perm] # superposed timestamps
u_i = u_i[perm]

### Reshape timestamps and marker data into the form of Steve Morse's EM code

In [23]:
data20 = []
counter = 0
for t,u in zip(t_i, u_i):
    data20.append([])
    data20[counter] += [t, np.array([u], dtype='int')]
    counter += 1


data20 = np.array(data20)
data20.shape

(6075, 2)

## Total number of observations for events of 20% price-drops were 6075

# Model 10 price series as Multivariate HP and apply EM algorithm to perform parameter estimation of the multivariate HP for 10% quantile

In [11]:
P_multiv10 = MHP.MHP() # instantiate MHP object
P_multiv10.generate_seq(20) # mu=[0.1], alpha=[[0.5]], and omega=1.0

Max eigenvalue: 0.50000


array([[8.351037185827542, array([0])],
       [11.752874639406835, array([0])]], dtype=object)

In [12]:
P_multiv10.data = data10
P_multiv10.data

array([[0.43677219292984165, array([4])],
       [0.6543285601112814, array([8])],
       [0.8872564525116836, array([7])],
       ...,
       [146.40395017603421, array([4])],
       [146.49618469324798, array([9])],
       [146.73634087005024, array([0])]], dtype=object)

In [16]:
seed = 99
rng = np.random.default_rng(None)
ahat = rng.uniform(0.1,1, size=(10,10))
mhat = rng.uniform(0.1,1, size=10)
w_hyperparam = np.linspace(0.5,5,100)

In [15]:
w_hyperparam

array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

In [17]:
startTime = time.time() # start timer

omega_index10 = 0 # store index of best value of omega 
ahat_vals10 = [] 
mhat_vals10 = []

ahat_arr10 = []; mhat_arr10 = []
LL_arr10 = []; res10 = []



for w in range(len(w_hyperparam)):
    #print(f'hyperparameter omega: ', w[i])

    res10 += [P_multiv10.EM(ahat, mhat, w_hyperparam[w], verbose=False)]
    ahat_arr10 += [res10[w][0]] 
    mhat_arr10 += [res10[w][1]]
    LL_arr10 += [-res10[w][-1]] # negative log-likelihood

executionTime = (time.time() - startTime) # stoper timer and measure the execution time
print('Execution time in minutes: ' + str(executionTime/60))

Execution time in minutes: 45.02928173144658


# Model 10 price series as Multivariate HP and apply EM algorithm to perform parameter estimation of the multivariate HP for 15% quantile

In [24]:
P_multiv15 = MHP.MHP() # instantiate MHP object
P_multiv15.generate_seq(20) # mu=[0.1], alpha=[[0.5]], and omega=1.0

Max eigenvalue: 0.50000


array([[2.8461913750448753, array([0])],
       [3.4172613513284866, array([0])],
       [11.578753719767796, array([0])]], dtype=object)

In [25]:
P_multiv15.data = data15
P_multiv15.data

array([[0.6405126057470842, array([4])],
       [0.7673582755676893, array([4])],
       [0.9169544297635996, array([7])],
       ...,
       [146.72738164110842, array([6])],
       [146.781537931665, array([2])],
       [146.9003790865634, array([0])]], dtype=object)

In [26]:
seed = 99
rng = np.random.default_rng(None)
ahat = rng.uniform(0.1,1, size=(10,10))
mhat = rng.uniform(0.1,1, size=10)
w_hyperparam = np.linspace(0.5,5,100)

In [27]:
startTime = time.time() # start timer

omega_index15 = 0 # store index of best value of omega 
ahat_vals15 = [] 
mhat_vals15 = []

ahat_arr15 = []; mhat_arr15 = []
LL_arr15 = []; res15 = []



for w in range(len(w_hyperparam)):
    #print(f'hyperparameter omega: ', w[i])

    res15 += [P_multiv15.EM(ahat, mhat, w_hyperparam[w], verbose=False)]
    ahat_arr15 += [res15[w][0]] 
    mhat_arr15 += [res15[w][1]]
    LL_arr15 += [-res15[w][-1]] # negative log-likelihood

executionTime = (time.time() - startTime) # stoper timer and measure the execution time
print('Execution time in minutes: ' + str(executionTime/60))

Execution time in minutes: 56.76680162747701


# Model 10 price series as Multivariate HP and apply EM algorithm to perform parameter estimation of the multivariate HP for 20% quantile

In [28]:
P_multiv20 = MHP.MHP() # instantiate MHP object
P_multiv20.generate_seq(20) # mu=[0.1], alpha=[[0.5]], and omega=1.0

Max eigenvalue: 0.50000


array([[8.100799527589691, array([0])],
       [14.266187292856682, array([0])],
       [14.308272620795698, array([0])],
       [14.998483326815665, array([0])],
       [15.71317953153696, array([0])],
       [16.358076062251705, array([0])],
       [17.058609391358402, array([0])],
       [17.186752806156218, array([0])]], dtype=object)

In [29]:
P_multiv20.data = data20
P_multiv20.data

array([[0.620751410205026, array([8])],
       [0.6221520395659299, array([7])],
       [0.6411493805084759, array([2])],
       ...,
       [146.77694603906556, array([9])],
       [146.79703879414006, array([8])],
       [146.91432801351917, array([2])]], dtype=object)

In [30]:
seed = 99
rng = np.random.default_rng(None)
ahat = rng.uniform(0.1,1, size=(10,10))
mhat = rng.uniform(0.1,1, size=10)
w_hyperparam20 = np.linspace(0.5,5,100)

In [31]:
startTime = time.time() # start timer

omega_index20 = 0 # store index of best value of omega 
ahat_vals20 = [] 
mhat_vals20 = []

ahat_arr20 = []; mhat_arr20 = []
LL_arr20 = []; res20 = []



for w in range(len(w_hyperparam)):
    #print(f'hyperparameter omega: ', w[i])

    res20 += [P_multiv20.EM(ahat, mhat, w_hyperparam20[w], verbose=False)]
    ahat_arr20 += [res20[w][0]] 
    mhat_arr20 += [res20[w][1]]
    LL_arr20 += [-res20[w][-1]] # negative log-likelihood

executionTime = (time.time() - startTime) # stoper timer and measure the execution time
print('Execution time in minutes: ' + str(executionTime/60))

Execution time in minutes: 94.45322011311849


In [32]:
# axes are in a two-dimensional array, indexed by [row, col]
#plt.rcdefaults()

fig, axs = plt.subplots(1, 3)

plt.rcParams["figure.figsize"] = (16, 9) #set default figure size (w, h) 
plt.style.use("ggplot")
# change the fontsize of the xtick and ytick labels
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)

# 10% case
min_index10 = np.argmin(LL_arr10)
omega_index10 = min_index10
mhat_vals10 = mhat_arr10[min_index10]
ahat_vals10 = ahat_arr10[min_index10]
omegahat10 = w_hyperparam[omega_index10]

# 15% case
min_index15 = np.argmin(LL_arr15)
omega_index15 = min_index15
mhat_vals15 = mhat_arr15[min_index15]
ahat_vals15 = ahat_arr15[min_index15]
omegahat15 = w_hyperparam[omega_index15]

# 20% case
min_index20 = np.argmin(LL_arr20)
omega_index20 = min_index20
mhat_vals20 = mhat_arr20[min_index20]
ahat_vals20 = ahat_arr20[min_index20]
omegahat20 = w_hyperparam[omega_index20]

axs[0].plot(w_hyperparam, LL_arr10)

axs[0].set_xlabel(r'omega, $\omega$',fontsize=20)

axs[0].set_ylabel('Negative log-likelihood',fontsize=20)
axs[0].scatter(x=w_hyperparam[min_index10],y=LL_arr10[min_index10], c='b',s=40,marker='X',linewidths=1)



axs[1].plot(w_hyperparam, LL_arr15)

axs[1].set_xlabel(r'omega, $\omega$',fontsize=20)

axs[1].scatter(x=w_hyperparam[min_index15],y=LL_arr15[min_index15], c='b',s=40,marker='X',linewidths=1)




axs[2].plot(w_hyperparam20, LL_arr20)

axs[2].set_xlabel(r'omega, $\omega$',fontsize=20)

axs[2].scatter(x=w_hyperparam20[min_index20],y=LL_arr20[min_index20], c='b',s=40,marker='X',linewidths=1)



plt.tight_layout()
#plt.savefig('NLL_multiv_unzoomed10_15_20.png')
plt.show()
plt.ion()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
omegahat10, omegahat15, omegahat20

(2.090909090909091, 2.3181818181818183, 2.1818181818181817)

In [6]:
mhat_vals10, mhat_vals15, mhat_vals20

(array([0.08323203, 0.11911056, 0.17495176, 0.17494162, 0.43068382,
        0.2258327 , 0.02641085, 0.14641441, 0.2785459 , 0.08151759]),
 array([0.03950351, 0.10116131, 0.05575779, 0.2398134 , 0.67374448,
        0.32636353, 0.01148297, 0.19069355, 0.09841817, 0.07592986]),
 array([0.01456292, 0.0770767 , 0.20278501, 0.18288805, 0.57295972,
        0.27609656, 0.07930378, 0.23417148, 0.0706239 , 0.03921634]))

In [24]:
squad

2.2727272727272725

# Save this session and load it back

In [33]:
import dill
dill.dump_session('DJIA_multivHP_10_15_20_September_v2_random10.db')

In [5]:
#dill.load_session('DJIA_multivHP_10_15_20_September_v1.db')
dill.dump_session('DJIA_multivHP_10_15_20_September_v2_random10.db')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
squad

# Save this session and load it back

In [ ]:
import dill
#dill.dump_session('DJIA_multivHP_15_env_v2.db')
#dill.load_session('DJIA_multivHP_env_v2.db')